## Семинар 1
# Оптимизация гиперпараметров

Гиперпараметр (англ. hyperparameter) — параметр, который не настраивается во время обучения модели. Пример гиперпараметра — шаг градиентного спуска, он задается перед обучением. Пример параметров — веса градиентного спуска, они изменяются и настраиваются во время обучения.

Попытаемся улучшить обобщающую способность модели, настроив ее гиперпараметры.
Задача оптимизации гиперпараметров заключается в подборе таких гиперпараметров, при которых заданная модель алгоритма будет наиболее эффективна.
Наиболее часто используемый метод – это решетчатый поиск (grid search), который перебирает все возможные комбинации интересующих гиперпараметров.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import datasets
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Задания для выполнения:

### 1. Сгенерируйте данные для задачи бинарной классификации с 4 признаками и 10000 наблюдениями. Целевая переменная должна зависеть от 2 признаков. Выведите признаки в виде датафрейма, дав названия колонкам. Выведите число объектов в каждом классе.
Указание: Для этого воспользуйтесь [make_classification](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html), задав параметры: число строк n_samples=10000, число признаков n_features=4, число признаков от которых зависит целевая переменная n_informative=2, число избыточных признаков (т.е. признаков, являющихся линейными комбинациями признаков от которых зависит целевая переменная) n_redundant=2, фиксируем воспроизводимость случайных данных random_state=42.

In [51]:
X, y = datasets.make_classification(n_samples=1000,
                           n_features=4,
                           n_informative=2,
                           n_redundant=2,
                           random_state=42)

In [52]:
X[:5]

array([[-1.60771127, -0.02939755,  1.56995261, -0.52798442],
       [ 0.29013139,  0.31768126, -0.99651948,  0.47700945],
       [-1.22509603,  0.89591149, -0.90032416,  0.71986083],
       [ 2.04686112, -1.68059502,  1.92371797, -1.42724398],
       [ 1.82888418, -0.28926129, -1.04915093,  0.20627144]])

In [53]:
df = pd.DataFrame(data = X, columns = ['X' + str(i) for i in range(1, X.shape[1] + 1)])
df

,X1,X2,X3,X4
0,-1.607711,-0.029398,1.569953,-0.527984
1,0.290131,0.317681,-0.996519,0.477009
2,-1.225096,0.895911,-0.900324,0.719861
3,2.046861,-1.680595,1.923718,-1.427244
4,1.828884,-0.289261,-1.049151,0.206271
...,...,...,...,...
995,1.399763,-1.159334,1.338486,-0.988308
996,-0.967111,0.994751,-1.367144,0.919603
997,0.710796,-0.612212,0.733345,-0.530584
998,2.029842,-0.416705,-0.946026,0.112039


In [54]:
pd.Series(y).value_counts()

,count
1,502
0,498


### 2. Разделите данные на обучающую и тестовую части. В тестовую часть отправьте 20% объектов и зафиксируйте способ перемешивания данных random_state=42. Выведите размеры обучающей и тестовой частей.

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [56]:
(X_train.shape, X_test.shape)

((800, 4), (200, 4))

### 3. Постройте модель логистической регрессии с гиперпараметрами, установленными по умолчанию. Для этого создайте класс LogisticRegression c дефолтными гиперпараметрами и обучите модель на обучающих данных. Сделайте предсказание на тестовых данных. Вычислите метрику accuracy на тестовых данных.

In [57]:
model_default = LogisticRegression()
model_default.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'deprecated',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [58]:
model_default.fit(X_train, y_train)

LogisticRegression()

In [59]:
y_pred = model_default.predict(X_test)
y_pred

array([1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 1])

In [60]:
accuracy = metrics.accuracy_score(y_test, y_pred)
accuracy

0.885

### 4. Напишите функцию, принимающую матрицу признаков и вектор целей и осуществляющую решетчатый поиск лучших гиперпараметров модели LogisticRegression по метрике accuracy с использованием hold-out разбиения, среди двух гиперпараметров, значения которых указанны ниже. Подберите лучшие гиперпараметры на обучающих данных. Обучите модель с лучшими гиперпараметрами на обучающих данных и вычислите метрику accuracy на тестовых данных.

Создадим диапазон вариантов для гиперпараметров в виде словаря:

In [61]:
np.logspace(-3, 2, 9)

array([1.00000000e-03, 4.21696503e-03, 1.77827941e-02, 7.49894209e-02,
       3.16227766e-01, 1.33352143e+00, 5.62341325e+00, 2.37137371e+01,
       1.00000000e+02])

In [62]:
param_grid = {'C': np.logspace(-3,2,9), 'solver': ['lbfgs','liblinear', 'saga'] }

In [63]:
def Grid(X,y):
  X_trainval, X_testval, y_trainval, y_testval = train_test_split(X, y, random_state=42, test_size=0.2)
  score = 0
  best_score = 0
  for C in param_grid['C']:
    for solver in param_grid['solver']:
      model = LogisticRegression(C = C, solver = solver)
      model.fit(X_trainval, y_trainval)
      score = model.score(X_testval, y_testval)
      if score > best_score:
        best_score = score
        best_parameters = {'C': C, 'solver': solver}

  return best_parameters

In [64]:
best_parameters = Grid(X_train, y_train)
best_parameters

{'C': 0.001, 'solver': 'liblinear'}

### 5. Напишите функцию, принимающую матрицу признаков и вектор целей и осуществляющую решетчатый поиск лучших гиперпараметров модели LogisticRegression по метрике accuracy с использованием перекрёстной проверки, для тех же гиперпараметров, что и в предыдущем задании. Подберите лучшие гиперпараметры на обучающих данных. Обучите модель с лучшими гиперпараметрами на обучающих данных и вычислите метрику accuracy на тестовых данных.

In [65]:
def GridCV(X,y):
  score, best_score = 0, 0
  for C in param_grid['C']:
    for solver in param_grid['solver']:
      model = LogisticRegression(C = C, solver = solver)
      score = np.mean(cross_val_score(model, X, y, cv=5, scoring="accuracy"))
      if score > best_score:
        best_score = score
        best_parameters = {'C': C, 'solver': solver}

  return best_parameters

In [66]:
best_parameters = GridCV(X_train, y_train)
best_parameters

{'C': 0.004216965034285823, 'solver': 'lbfgs'}

In [67]:
best_model = LogisticRegression(**best_parameters)
best_model.fit(X_train, y_train)
best_accuracy = best_model.score(X_test, y_test)
best_accuracy

0.87

### 6. Осуществите решетчатый поиск на обучающих данных лучших гиперпараметров модели LogisticRegression по метрике accuracy с использованием перекрёстной проверки, для тех же гиперпараметров, что и в предыдущем задании, используя класс GridSearchCV из библиотеки sklearn. Убедитесь, что результаты будут теми же, что и в предыдущем задании.
Указание: Для выведения наилучших гиперпараметров модели, используйте атрибут best_params_. Наилучшая модель выводится при помощи атрибута best_estimator_.

In [68]:
gs = GridSearchCV(
    estimator = LogisticRegression(),
    param_grid =  param_grid,
    scoring = 'accuracy',
    cv = 5
)

In [69]:
gs.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': array([1.00000000e-03, 4.21696503e-03, 1.77827941e-02, 7.49894209e-02,
       3.16227766e-01, 1.33352143e+00, 5.62341325e+00, 2.37137371e+01,
       1.00000000e+02]),
                         'solver': ['lbfgs', 'liblinear', 'saga']},
             scoring='accuracy')

In [70]:
gs.best_params_

{'C': 0.004216965034285823, 'solver': 'lbfgs'}

In [71]:
best_model_skl = LogisticRegression(**gs.best_params_)
best_model_skl.fit(X_train, y_train)
best_model_skl.score(X_test, y_test)

0.87

In [72]:
#или
best_model_skl = gs.best_estimator_
best_model_skl.fit(X_train, y_train)
best_model_skl.score(X_test, y_test)

0.87

### 7. Выведите результаты решетчатого поиска с использованием класса GridSearchCV, осуществлённого в предыдущем пункте, в виде словаря и в виде датафрейма. Оставьте в датафрейме только два столбца со значениями гиперпараметров и со значениями вычисляемой метрики. Выведите самое большое значение среди средних значений метрики accuracy перекрестной проверки.
Указание: Для выведения результатов решетчатого поиска в виде словаря используйте атрибут cv_results_. Для выведения самого большого значения среди средних значений метрики accuracy используйте атрибут best_score_.

In [73]:
pd.DataFrame(gs.cv_results_).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003300,0.000372,0.001241,0.000075,0.001000,lbfgs,"{'C': 0.001, 'solver': 'lbfgs'}",0.83750,0.88750,0.8750,0.81875,0.88750,0.86125,0.028062,3
1,0.001661,0.000045,0.001158,0.000068,0.001000,liblinear,"{'C': 0.001, 'solver': 'liblinear'}",0.82500,0.88125,0.8750,0.81875,0.88125,0.85625,0.028229,12
2,0.003623,0.000224,0.001146,0.000016,0.001000,saga,"{'C': 0.001, 'solver': 'saga'}",0.83750,0.88750,0.8750,0.81875,0.88750,0.86125,0.028062,3
3,0.003547,0.000193,0.001284,0.000022,0.004217,lbfgs,"{'C': 0.004216965034285823, 'solver': 'lbfgs'}",0.83125,0.89375,0.8625,0.83125,0.89375,0.86250,0.027951,1
4,0.001794,0.000204,0.001343,0.000029,0.004217,liblinear,"{'C': 0.004216965034285823, 'solver': 'libline...",0.82500,0.87500,0.8750,0.83125,0.88750,0.85875,0.025495,8


In [74]:
pd.DataFrame({'params': gs.cv_results_['params'], 'mean_test_score': gs.cv_results_['mean_test_score']}).head()

,params,mean_test_score
0,"{'C': 0.001, 'solver': 'lbfgs'}",0.86125
1,"{'C': 0.001, 'solver': 'liblinear'}",0.85625
2,"{'C': 0.001, 'solver': 'saga'}",0.86125
3,"{'C': 0.004216965034285823, 'solver': 'lbfgs'}",0.86250
4,"{'C': 0.004216965034285823, 'solver': 'libline...",0.85875


In [75]:
gs.best_score_.round(6)

0.8625

### 8. Используйте методы score и predict класса GridSearchCV, чтобы вычислить метрики качества accuracy и F1-меры на тестовых данных для модели LogisticRegression с лучшими гиперпараметрами.

In [76]:
gs.score(X_test, y_test)

0.87

In [77]:
y_test_pred = gs.predict(X_test)
metrics.f1_score(y_test, y_test_pred)

0.8617021276595744

### 9. Осуществите решетчатый поиск на обучающих данных лучших гиперпараметров модели LogisticRegression по метрике accuracy с использованием перекрёстной проверки, используя класс GridSearchCV из библиотеки sklearn, среди списка словарей гиперпараметров, указанных ниже. Обучите модель с лучшими гиперпараметрами на обучающих данных и вычислите метрики accuracy и F1-меру на тестовых данных.

In [78]:
param_grid = [{'solver': ['liblinear'],
    'penalty': ['l1', 'l2'],
    'C': np.logspace(-3,2,9),
    'max_iter' : [1000]
   }, {
    'solver': ['saga'],
    'penalty': ['elasticnet'],
    'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],
    'max_iter' : [1000]
}]

In [79]:
gs_l = GridSearchCV(
    estimator = LogisticRegression(),
    param_grid =  param_grid,
    scoring = 'accuracy',
    cv = 5
)

In [80]:
gs_l.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid=[{'C': array([1.00000000e-03, 4.21696503e-03, 1.77827941e-02, 7.49894209e-02,
       3.16227766e-01, 1.33352143e+00, 5.62341325e+00, 2.37137371e+01,
       1.00000000e+02]),
                          'max_iter': [1000], 'penalty': ['l1', 'l2'],
                          'solver': ['liblinear']},
                         {'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],
                          'max_iter': [1000], 'penalty': ['elasticnet'],
                          'solver': ['saga']}],
             scoring='accuracy')

In [81]:
gs_l.best_params_

{'C': 0.01778279410038923,
 'max_iter': 1000,
 'penalty': 'l2',
 'solver': 'liblinear'}

In [82]:
gs_l.score(X_test, y_test)

0.875

### 10. Осуществите рандомизированный решетчатый поиск на обучающих данных лучших гиперпараметров модели LogisticRegression по метрике accuracy с использованием перекрёстной проверки, используя класс RandomizedSearchCV из библиотеки sklearn, для списка словарей гиперпараметров из предыдущего задания. Обучите модель с лучшими гиперпараметрами на обучающих данных и вычислите метрики accuracy и F1-меру на тестовых данных.

In [83]:
rgs = RandomizedSearchCV(
    estimator = LogisticRegression(),
    param_distributions =  param_grid,
    scoring = 'accuracy',
    cv = 5,
    random_state = 1
)

In [84]:
rgs.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=LogisticRegression(),
                   param_distributions=[{'C': array([1.00000000e-03, 4.21696503e-03, 1.77827941e-02, 7.49894209e-02,
       3.16227766e-01, 1.33352143e+00, 5.62341325e+00, 2.37137371e+01,
       1.00000000e+02]),
                                         'max_iter': [1000],
                                         'penalty': ['l1', 'l2'],
                                         'solver': ['liblinear']},
                                        {'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],
                                         'max_iter': [1000],
                                         'penalty': ['elasticnet'],
                                         'solver': ['saga']}],
                   random_state=1, scoring='accuracy')

In [85]:
pd.DataFrame({'params': rgs.cv_results_['params'], 'mean_test_score': rgs.cv_results_['mean_test_score']}).head()

,params,mean_test_score
0,"{'solver': 'saga', 'penalty': 'elasticnet', 'm...",0.85500
1,"{'solver': 'liblinear', 'penalty': 'l2', 'max_...",0.85500
2,"{'solver': 'liblinear', 'penalty': 'l2', 'max_...",0.85875
3,"{'solver': 'liblinear', 'penalty': 'l2', 'max_...",0.85500
4,"{'solver': 'saga', 'penalty': 'elasticnet', 'm...",0.85500


In [86]:
rgs.best_params_

{'solver': 'liblinear',
 'penalty': 'l2',
 'max_iter': 1000,
 'C': 0.004216965034285823}

In [87]:
rgs.score(X_test, y_test)

0.87

# Задания для самостоятельного выполнения:

### 1. Загрузите встроенный датасет `load_iris`. Обозначьте признаки за `X`, а целевую переменную за `y`. Выведите первые 5 строк в виде датафрейма.

In [1]:
from sklearn.datasets import  load_iris

In [2]:
iris = load_iris()

In [6]:
X = iris.data
y = iris.target

In [7]:
df = pd.DataFrame(X, columns=iris.feature_names)
df["target"] = y
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


### 2. Разделите данные на обучающую и тестовую части. В тестовую часть отправьте 15% объектов и зафиксируйте способ перемешивания данных random_state=0. Выведите размеры обучающей и тестовой частей.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.15)

In [9]:
X_train.shape, y_train.shape

((127, 4), (127,))

### 3. Для датасета load_iris осуществите классификацию методом логистической регрессии, создав класс LogisticRegression(max_iter = 1000). Выведите значения гиперпараметров этой модели, установленные по умолчанию. Обучите эту модель, сделайте предсказание. Оцените качество модели, выведя значения метрик accuracy и f1-score.

In [10]:
model_default = LogisticRegression(max_iter = 1000)
model_default.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 1000,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [11]:
model_default.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [12]:
y_pred = model_default.predict(X_test)
y_pred

array([2, 1, 0, 2, 0, 2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0, 2, 1,
       0])

In [13]:
accuracy = metrics.accuracy_score(y_test, y_pred)
accuracy

1.0

In [15]:
f1 = metrics.f1_score(y_test, y_pred, average='weighted')
f1

1.0

### 4. Оптимизируйте гиперпараметры, используя следующие наборы гиперпараметров: param_grid = [{'solver': ['lbfgs', 'newton-cg'],'penalty': ['l2']}, {'C': np.logspace(0,4,10), 'penalty': ['l2', 'l1'], 'solver': ['liblinear']}, {'C': np.logspace(0,4,10), 'penalty': ['l2'], 'solver': ['lbfgs', 'newton-cg']},] по метрике accuracy. Реализуйте поиск лучших гиперпараметров с перекрёстной проверкой, используя класс GridSearchCV. Выведете результаты поиска в виде датафрейма, состоящего из двух колонок: в первой колонке все перебираемые сочитания гиперпараметров, а во второй соответствующие им средние значения метрики accuracy. Выведите значения лучших гиперпараметров.

In [18]:
param_grid = [{'solver': ['lbfgs', 'newton-cg'],'penalty': ['l2']}, {'C': np.logspace(0,4,10), 'penalty': ['l2', 'l1'], 'solver': ['liblinear']}, {'C': np.logspace(0,4,10), 'penalty': ['l2'], 'solver': ['lbfgs', 'newton-cg']},]

In [19]:
gs = GridSearchCV(
    estimator = LogisticRegression(max_iter = 1000),
    param_grid =  param_grid,
    scoring = 'accuracy',
    cv = 5
)

In [20]:
gs.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=1000),
             param_grid=[{'penalty': ['l2'], 'solver': ['lbfgs', 'newton-cg']},
                         {'C': array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
       5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
       3.59381366e+03, 1.00000000e+04]),
                          'penalty': ['l2', 'l1'], 'solver': ['liblinear']},
                         {'C': array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
       5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
       3.59381366e+03, 1.00000000e+04]),
                          'penalty': ['l2'],
                          'solver': ['lbfgs', 'newton-cg']}],
             scoring='accuracy')

In [21]:
gs.best_params_

{'C': 21.544346900318832, 'penalty': 'l1', 'solver': 'liblinear'}

### 5. Выведите значение метрики accuracy на тестовых данных. Cделайте предсказание для лучшей модели и выведите метрику f1-score. Сравните полученные метрики с метриками, полученными без оптимизации гиперпараметров.

In [23]:
best_model_skl = gs.best_estimator_
best_model_skl.score(X_test, y_test)

0.9565217391304348

In [24]:
y_test_pred = gs.predict(X_test)
metrics.f1_score(y_test, y_test_pred, average='weighted')

0.9550514597773576

### 6. Осуществите рандомизированный решетчатый поиск лучших гиперпараметров по метрике accuracy для той же самой сетки гиперпараметров.

In [25]:
rgs = RandomizedSearchCV(
    estimator = LogisticRegression(max_iter=1000),
    param_distributions =  param_grid,
    scoring = 'accuracy',
    cv = 5,
    random_state = 1
)

In [26]:
rgs.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=LogisticRegression(max_iter=1000),
                   param_distributions=[{'penalty': ['l2'],
                                         'solver': ['lbfgs', 'newton-cg']},
                                        {'C': array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
       5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
       3.59381366e+03, 1.00000000e+04]),
                                         'penalty': ['l2', 'l1'],
                                         'solver': ['liblinear']},
                                        {'C': array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
       5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
       3.59381366e+03, 1.00000000e+04]),
                                         'penalty': ['l2'],
                                         'solver': ['lbfgs', 'newton-cg']}],
                   random_state=1, scoring='accuracy')

In [27]:
rgs.best_params_

{'solver': 'newton-cg', 'penalty': 'l2', 'C': 3593.813663804626}

In [28]:
rgs.score(X_test, y_test)

1.0

### 7. Осуществите решетчатый поиск лучших гиперпараметров по метрике f1 для той же самой сетки гиперпараметров.

In [33]:
rgs = RandomizedSearchCV(
    estimator = LogisticRegression(max_iter=1000),
    param_distributions =  param_grid,
    scoring = 'f1_weighted',
    cv = 5,
    random_state = 1
)

In [34]:
rgs.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=LogisticRegression(max_iter=1000),
                   param_distributions=[{'penalty': ['l2'],
                                         'solver': ['lbfgs', 'newton-cg']},
                                        {'C': array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
       5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
       3.59381366e+03, 1.00000000e+04]),
                                         'penalty': ['l2', 'l1'],
                                         'solver': ['liblinear']},
                                        {'C': array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
       5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
       3.59381366e+03, 1.00000000e+04]),
                                         'penalty': ['l2'],
                                         'solver': ['lbfgs', 'newton-cg']}],
                   random_state=1, scoring='f1_weighted')

In [39]:
best_model_skl = rgs.best_estimator_
y_pred = rgs.predict(X_test)
metrics.f1_score(y_test, y_pred, average='weighted')

1.0